In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('asd.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
data = data[data.sentiment != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)

    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

267694
289836


In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 54, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.50, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(139382, 54) (139382, 2)
(139383, 54) (139383, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs =40 , batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
 - 773s - loss: 0.2484 - acc: 0.8918
Epoch 2/40
 - 858s - loss: 0.1694 - acc: 0.9295
Epoch 3/40
 - 857s - loss: 0.1501 - acc: 0.9383
Epoch 4/40
 - 857s - loss: 0.1367 - acc: 0.9451
Epoch 5/40
 - 855s - loss: 0.1275 - acc: 0.9486
Epoch 6/40
 - 854s - loss: 0.1188 - acc: 0.9521
Epoch 7/40
 - 861s - loss: 0.1130 - acc: 0.9546
Epoch 8/40
 - 858s - loss: 0.1057 - acc: 0.9576
Epoch 9/40
 - 857s - loss: 0.1003 - acc: 0.9601
Epoch 10/40
 - 856s - loss: 0.0954 - acc: 0.9621
Epoch 11/40
 - 857s - loss: 0.0913 - acc: 0.9637
Epoch 12/40
 - 856s - loss: 0.0882 - acc: 0.9644
Epoch 13/40
 - 861s - loss: 0.0851 - acc: 0.9659
Epoch 14/40
 - 858s - loss: 0.0813 - acc: 0.9677
Epoch 15/40
 - 855s - loss: 0.0787 - acc: 0.9687
Epoch 16/40
 - 855s - loss: 0.0780 - acc: 0.9689
Epoch 17/40
 - 855s - loss: 0.0750 - acc: 0.9700
Epoch 18/40
 - 855s - loss: 0.0738 - acc: 0.9706
Epoch 19/40
 - 853s - loss: 0.0728 - acc: 0.9713
Epoch 20/40
 - 855s - loss: 0.

In [7]:
validation_size = 68000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.19
acc: 0.95


In [8]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 94.02861098292571 %
neg_acc 94.66403718833638 %


In [9]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=54, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 135   5  44   8  29  29  31   5  44]]
negative
